In [1]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import warnings
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils.multiclass import unique_labels
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('ds_challenge_data_final.csv',sep='\t')
#df.fillna('none', inplace = True)
df.fillna(0, inplace = True)
#df = df.sample(frac=0.35)
df.columns.to_list()

In [3]:
df['holiday_nm'] = df['holiday_nm'].apply(lambda x: 'none' if ((str(x).strip()=='0') | (x==0)) else x)

In [4]:
pred_cat_cols = ['holiday_nm']
cols_to_drop = ['Transaction_Date','Hash_Trans_ID','PRO' ]

In [5]:
df = pd.get_dummies(data=df, columns = pred_cat_cols)

In [6]:
df_train = df[df['Transaction_Year'] < 2015]
train_targets = df_train['PRO']
train_data = df_train.drop(cols_to_drop, axis=1, inplace=False)

In [7]:
mdl = xgb.XGBClassifier(max_depth=2, min_child_weight=1, reg_alpha=1e-5, gamma=0,subsample=0.9, colsample_bytree=0.75, learning_rate=0.2, n_estimators=250, verbosity=1, n_jobs=-1)

In [14]:
n=2

scores=pd.DataFrame(np.zeros([n,train_data.shape[1]]))
scores.columns=train_data.columns
ct=0

for train_idx, test_idx in ShuffleSplit(n_splits=n, train_size=0.75, test_size=.25).split(train_data):
    ct+=1
    X_train = train_data.ix[train_idx,:]
    X_test = train_data.ix[test_idx,:]
    Y_train = train_targets.ix[train_idx]
    Y_test = train_targets.ix[test_idx]
    print('training model {}'.format(ct))
    r = mdl.fit(X_train, Y_train)
    fpr, tpr, thresholds = metrics.roc_curve(Y_test.astype('int'), np.array(mdl.predict_proba(X_test))[:,1], pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print('auc is {}'.format(auc))
    for i in range(train_data.shape[1]):
        X_t = X_test.copy()
        X_t.ix[:,i]=shuffle(np.array(X_t.ix[:, i]))
        fpr, tpr, thresholds = metrics.roc_curve(Y_test.astype('int'), np.array(mdl.predict_proba(X_t))[:,1], pos_label=1)
        shuff_auc = metrics.auc(fpr, tpr)
        scores.ix[ct-1,i]=((auc-shuff_auc)/auc)

training model 1
auc is 0.7013383451434196
training model 2
auc is 0.7062030908841104


In [15]:
fin_score=pd.DataFrame(np.zeros([train_data.shape[1],4]))
fin_score.columns=['Mean','Median','Max','Min']
fin_score.index=train_data.columns
fin_score.ix[:,0]=scores.mean()
fin_score.ix[:,1]=scores.median()
fin_score.ix[:,2]=scores.min()
fin_score.ix[:,3]=scores.max()

In [ ]:
#output cleared
pd.set_option('display.max_rows', None)
fin_score.sort_values('Mean',axis=0, ascending=False)

### Run the baseline model

In [17]:
cols_to_drop = ['Transaction_Date','Hash_Trans_ID','PRO' ]

In [18]:
df_train = df[df['Transaction_Year'] < 2015]
train_targets = df_train['PRO'].as_matrix().astype('float32')
train_data = df_train.drop(cols_to_drop, axis=1, inplace=False).as_matrix().astype('float32')
df_test = df[df['Transaction_Year'] == 2015]
test_targets = df_test['PRO'].as_matrix().astype('float32')
test_data = df_test.drop(cols_to_drop, axis=1, inplace=False).as_matrix().astype('float32')

In [19]:
mdl.fit(train_data,train_targets)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=0.75, gamma=0, learning_rate=0.2,
              max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
              n_estimators=250, n_jobs=-1, nthread=None,
              objective='binary:logistic', random_state=0, reg_alpha=1e-05,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
              subsample=0.9, verbosity=1)

In [20]:
test_pred = mdl.predict_proba(test_data)
predictions = np.array(test_pred)[:,1]
actuals = test_targets.astype('int')
fpr, tpr, thresholds = metrics.roc_curve(actuals, predictions, pos_label=1)
metrics.auc(fpr, tpr)

0.7188943450700578

### Now drop the unwanted columns

In [21]:
cols_to_drop = ['Transaction_Date','Hash_Trans_ID','PRO']

In [22]:
d = fin_score.reset_index().rename(columns={'index':'col_name'})

In [23]:
cols_to_drop = cols_to_drop + d[d['Mean']<0]['col_name'].to_list()

In [24]:
df_train = df[df['Transaction_Year'] < 2015]
train_targets = df_train['PRO'].as_matrix().astype('float32')
train_data = df_train.drop(cols_to_drop, axis=1, inplace=False).as_matrix().astype('float32')
df_test = df[df['Transaction_Year'] == 2015]
test_targets = df_test['PRO'].as_matrix().astype('float32')
test_data = df_test.drop(cols_to_drop, axis=1, inplace=False).as_matrix().astype('float32')

In [25]:
mdl.fit(train_data,train_targets)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=0.75, gamma=0, learning_rate=0.2,
              max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
              n_estimators=250, n_jobs=-1, nthread=None,
              objective='binary:logistic', random_state=0, reg_alpha=1e-05,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
              subsample=0.9, verbosity=1)

In [26]:
test_pred = mdl.predict_proba(test_data)
predictions = np.array(test_pred)[:,1]
actuals = test_targets.astype('int')
fpr, tpr, thresholds = metrics.roc_curve(actuals, predictions, pos_label=1)
metrics.auc(fpr, tpr)

0.7163284576900226

## Save the wanted features only

In [27]:
cols_to_drop = d[d['Mean']<0]['col_name'].to_list()

In [28]:
df.drop(cols_to_drop, axis=1, inplace=True)

In [29]:
df.to_csv('ds_challenge_data_impfeat.csv', sep='\t' ,index=False)